In [17]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import pickle
import os
import missingno as msno
import shapely.io
from shapely.geometry import Point, LineString

import base64
from shapely import wkb

from lxml import etree
from datetime import time

# Load red bicicleta

In [3]:
import json

with open('data/redes/red_bicicleta_normatividad.txt', 'r') as file:
    bici = json.load(file)

In [4]:
bici.keys()

In [5]:
bici['graph']

## Load nodes

In [6]:
bici['nodes'][0].keys()

In [7]:
x = []
y = []
id = []

for node in bici['nodes']:
    x.append(node['x'])
    y.append(node['y'])
    id.append(node['id'])


In [8]:
nodes_geometry = gpd.GeoSeries([Point(x, y) for x, y in zip(x, y)], name='geometry')

nodes = gpd.GeoDataFrame({'geometry': nodes_geometry, 'id': id})
nodes.plot()

In [9]:
nodes = nodes.set_crs('ETRS89_UTM_zone_30N')

## Load links

In [10]:
bici['links'][0].keys()

In [11]:
bici['links'][0]

In [12]:
bici['links'][0]['geometry']['py/reduce'][1]['py/tuple'][0]['py/b64']

In [13]:
import base64
from shapely import wkb

osmid = []
length = []
time = []
geometry = []
source = []
target = []
key = []

for link in bici['links']:
    osmid.append(link['osmid'])
    length.append(link['length'])
    time.append(link['time'])
    
    geometry_base64 = link['geometry']['py/reduce'][1]['py/tuple'][0]['py/b64']
    
    # Step 1: Decode the Base64 string into binary data
    binary_data = base64.b64decode(geometry_base64)
    
    # Step 2: Deserialize the binary data using pickle
    geometry_shapely = wkb.loads(binary_data, hex=True)
    
    geometry.append(geometry_shapely)
    
    source.append(link['source'])
    target.append(link['target'])
    key.append(link['key'])



In [14]:
links = gpd.GeoDataFrame({'osmid': osmid, 
                          'length':length,
                          'time':time,
                          'geometry':geometry,
                          'source':source,
                          'target':target,
                          'key':key
                          })
links.plot()

In [15]:
links = links.set_crs('ETRS89_UTM_zone_30N')

## Save to SHP

In [16]:
nodes.to_file('data/redes/red_bicicleta_normatividad_nodes.shp', driver='ESRI Shapefile')
links.to_file('data/redes/red_bicicleta_normatividad_links.shp', driver='ESRI Shapefile')

# Load red caminable

In [3]:
import json

with open('data/redes/caminable/red_caminable_normatividad.txt', 'r') as file:
    caminable = json.load(file)

In [4]:
caminable.keys()

In [5]:
caminable['graph']

## Load nodes

In [7]:
caminable['nodes'][0].keys()

In [10]:
caminable['nodes'][0]['highway']

In [21]:
x = []
y = []
highway = []
street_count = []
lat = []
lon = []
id = []

for node in caminable['nodes']:
    # x.append(node['x'])
    
    x_base64 = node['x']['py/reduce'][1]['py/tuple'][0]['py/b64']
    binary_data = base64.b64decode(x_base64)
    x_shapely = wkb.loads(binary_data, hex=True)
    x.append(x_shapely)
    
    y.append(node['y'])
    # highway.append(node['highway'])
    street_count.append(node['street_count'])
    lat.append(node['lat'])
    lon.append(node['lon'])
    id.append(node['id'])


In [14]:
y

In [12]:
caminable_nodes_geometry = gpd.GeoSeries([Point(x, y) for x, y in zip(x, y)], name='geometry')

caminable_nodes = gpd.GeoDataFrame({'geometry': caminable_nodes_geometry, 'id': id})
caminable_nodes.plot()

In [9]:
nodes = nodes.set_crs('ETRS89_UTM_zone_30N')

## Load links

In [10]:
bici['links'][0].keys()

In [11]:
bici['links'][0]

In [12]:
bici['links'][0]['geometry']['py/reduce'][1]['py/tuple'][0]['py/b64']

In [13]:
import base64
from shapely import wkb

osmid = []
length = []
time = []
geometry = []
source = []
target = []
key = []

for link in bici['links']:
    osmid.append(link['osmid'])
    length.append(link['length'])
    time.append(link['time'])
    
    geometry_base64 = link['geometry']['py/reduce'][1]['py/tuple'][0]['py/b64']
    
    # Step 1: Decode the Base64 string into binary data
    binary_data = base64.b64decode(geometry_base64)
    
    # Step 2: Deserialize the binary data using pickle
    geometry_shapely = wkb.loads(binary_data, hex=True)
    
    geometry.append(geometry_shapely)
    
    source.append(link['source'])
    target.append(link['target'])
    key.append(link['key'])



In [14]:
links = gpd.GeoDataFrame({'osmid': osmid, 
                          'length':length,
                          'time':time,
                          'geometry':geometry,
                          'source':source,
                          'target':target,
                          'key':key
                          })
links.plot()

# Load `centros_docentes_12-11-2024.xls`

In [11]:
centros = pd.read_excel('data/centros_docentes_12-11-2024.xls')
centros

In [12]:
msno.matrix(centros)

# Load `resumen-de-escolarizacion_2023.csv`

In [14]:
escolarizacion = pd.read_csv('data/resumen-de-escolarizacion_2023.csv', sep=';')
escolarizacion

In [15]:
msno.matrix(escolarizacion)